In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
# Initialize Mediapipe Hand Tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize canvas and default settings
canvas = None
drawing_color = (255, 255, 255)  # Default color: white
brush_thickness = 5  # Default brush size
prev_x, prev_y = None, None

# Start webcam capture
cap = cv2.VideoCapture(0)

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Mirror the frame for better UX
        h, w, _ = frame.shape

        # Initialize canvas once
        if canvas is None:
            canvas = np.zeros((h, w, 3), dtype=np.uint8)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        # Check if any hands are detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Get the coordinates of the index finger tip
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

                # Draw line from previous to current point if we have a previous point
                if prev_x is not None and prev_y is not None:
                    cv2.line(canvas, (prev_x, prev_y), (x, y), drawing_color, brush_thickness)

                # Update previous points
                prev_x, prev_y = x, y

                # Optional: Display hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        else:
            prev_x, prev_y = None, None  # Reset previous points if no hand is detected

        # Blend the canvas with the frame
        combined_frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
        cv2.putText(combined_frame, "Press 'c' to clear, 's' to save, 'q' to quit", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the result
        cv2.imshow('Air Drawing', combined_frame)

        # Handle keypresses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Quit
            break
        elif key == ord('c'):  # Clear the canvas
            canvas = np.zeros((h, w, 3), dtype=np.uint8)
        elif key == ord('s'):  # Save the canvas
            cv2.imwrite('air_drawing_output.png', canvas)
            print("Canvas saved as 'air_drawing_output.png'")
        elif key == ord('r'):  # Change color to red
            drawing_color = (0, 0, 255)
        elif key == ord('g'):  # Change color to green
            drawing_color = (0, 255, 0)
        elif key == ord('b'):  # Change color to blue
            drawing_color = (255, 0, 0)
        elif key == ord('w'):  # Change color to white
            drawing_color = (255, 255, 255)
        elif key == ord('+'):  # Increase brush size
            brush_thickness = min(50, brush_thickness + 2)
        elif key == ord('-'):  # Decrease brush size
            brush_thickness = max(2, brush_thickness - 2)

finally:
    cap.release()
    cv2.destroyAllWindows()